# Week 7.1 Recurrent Neural Networks

We'll being using **word embeddings** as inputs, so if you've already downloaded them in Week 5, **DONT DO IT AGAIN**. 



In [1]:
import numpy as np  # Keras takes care of most of this but it likes to see Numpy arrays
from keras.preprocessing import sequence    # A helper module to handle padding input
from keras.models import Sequential         # The base keras Neural Network model
from keras.layers import Dense, Dropout, Activation   # The layer objects we will pile into the model
from gensim.models.keyedvectors import KeyedVectors
from nltk.tokenize.casual import casual_tokenize
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
import pandas as pd
from random import shuffle
!git clone https://github.com/Louismac/NLP-Public
%cd NLP-Public
#Code editted from NLPIA book ch8 https://github.com/totalgood/nlpia#

# Colab users

In [ ]:
#Run this cell ONLY IF you're on colab
#This is going to download a giant file into your colab distribution
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
embeddings_file = '/root/input/GoogleNews-vectors-negative300.bin.gz' # from above


In [2]:
#This line will load the 200,000 most common words into wv, a variable of word vectors
#Note that this will load these into memory, so if you don't have a lot of memory on your computer you may run into problems/slowness
#embeddings_file = "GoogleNews-vectors-negative300.bin.gz"
wv = KeyedVectors.load_word2vec_format(embeddings_file, binary=True, limit=200000)

In [3]:
word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True, limit=200000)
def tokenize_and_vectorize(dataset):
    vectorized_data = []
    for sample in dataset:
        tokens = casual_tokenize(sample)
        sample_vecs = []
        for token in tokens:
            try:
                sample_vecs.append(word_vectors[token])
            except KeyError:
                pass  # No matching token in the Google w2v vocab
        vectorized_data.append(sample_vecs)

    return vectorized_data

def pad_trunc(data, maxlen):
    """ For a given dataset pad with zero vectors or truncate to maxlen """
    new_data = []

    # Create a vector of 0's the length of our word vectors
    zero_vector = []
    for _ in range(len(data[0][0])):
        zero_vector.append(0.0)

    for sample in data:
 
        if len(sample) > maxlen:
            temp = sample[:maxlen]
        elif len(sample) < maxlen:
            temp = sample
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
                temp.append(zero_vector)
        else:
            temp = sample
        new_data.append(temp)
    return new_data

# Classification 

In [4]:
file_name = "galleryorrecordlabel"
dataset = pd.read_csv("data/" + file_name + ".tsv",sep = "\t")
# shuffle the DataFrame rows 
dataset = dataset.sample(frac = 1) 
features = tokenize_and_vectorize(dataset["text"])
x_train, x_test, y_train, y_test = train_test_split(features, dataset["label"], test_size=0.3, random_state=0)

In [5]:
maxlen = 50
embedding_dims = 300    #

In [6]:
np.array(x_train).shape,np.array(x_test).shape

((4986,), (2138,))

In [7]:
x_train = pad_trunc(x_train, maxlen)
x_test = pad_trunc(x_test, maxlen)
x_train = np.reshape(x_train, (len(x_train), maxlen, embedding_dims))
y_train = np.array(y_train)
x_test = np.reshape(x_test, (len(x_test), maxlen, embedding_dims))
y_test = np.array(y_test)

In [8]:
np.array(x_train).shape,np.array(x_test).shape

((4986, 50, 300), (2138, 50, 300))

In [11]:
batch_size = 32        # How many samples to show the net before backpropogating the error and updating the weights      # The width of the filters, actual filters will each be a matrix of weights of size: embedding_dims x kernel_size or 50 x 3 in our case
num_neurons = 10     # Number of neurons in the plain feed forward net at the end of the chain
epochs = 5    

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, SimpleRNN, LSTM

print('Build model...')
model = Sequential()

model.add(LSTM(num_neurons, return_sequences=False, input_shape=(maxlen, embedding_dims)))
model.add(Dropout(.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile('rmsprop', 'binary_crossentropy',  metrics=['accuracy'])
print(model.summary())

Build model...
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10)                12440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 12,451
Trainable params: 12,451
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))
model_structure = model.to_json()
with open("simplernn_model2.json", "w") as json_file:
    json_file.write(model_structure)

model.save_weights("simplernn_weights2.h5")
print('Model saved.')

Epoch 1/5
156/156 [==============================] - 3s 16ms/step - loss: 0.4846 - accuracy: 0.7834 - val_loss: 0.3274 - val_accuracy: 0.8868
Epoch 2/5
156/156 [==============================] - 2s 14ms/step - loss: 0.3050 - accuracy: 0.9021 - val_loss: 0.2694 - val_accuracy: 0.9097
Epoch 3/5
156/156 [==============================] - 2s 14ms/step - loss: 0.2560 - accuracy: 0.9194 - val_loss: 0.2614 - val_accuracy: 0.9097
Epoch 4/5
156/156 [==============================] - 2s 14ms/step - loss: 0.2348 - accuracy: 0.9242 - val_loss: 0.2425 - val_accuracy: 0.9130
Epoch 5/5
156/156 [==============================] - 2s 14ms/step - loss: 0.2208 - accuracy: 0.9282 - val_loss: 0.2240 - val_accuracy: 0.9224
Model saved.
